In [ ]:
import cv2
import os
import glob
import math
import numpy as np
import tifffile as tiff
from PIL import Image
from pathlib import Path
from itertools import cycle, islice

In [ ]:
def calculate_ndvi(red_img_path: str, nir_img_path: str):

    red_image = np.asarray(Image.open(red_img_path)).astype(np.float64)
    nir_image = np.asarray(Image.open(nir_img_path)).astype(np.float64)

    numerator = nir_image - red_image
    denominator = nir_image + red_image

    ndvi = np.divide(numerator, denominator, out=np.zeros_like(
        numerator), where=denominator != 0)

    # normalize the image to range 0 to 255 and convert to uint8
    ndvi_norm = (ndvi + 1) / 2
    img_uint8 = (ndvi_norm * 255).astype(np.uint8)

    return img_uint8

def calculate_ndre(re_img_path: str, nir_img_path: str):

    re_image = np.asarray(Image.open(re_img_path)).astype(np.float64)
    nir_image = np.asarray(Image.open(nir_img_path)).astype(np.float64)

    numerator = nir_image - re_image
    denominator = nir_image + re_image

    ndre = np.divide(numerator, denominator, out=np.zeros_like(
        numerator), where=denominator != 0)

    # normalize the image to range 0 to 255 and convert to uint8
    ndre_norm = (ndre + 1) / 2
    img_uint8 = (ndre_norm * 255).astype(np.uint8)

    return img_uint8

In [ ]:
batch_path = Path("/mnt/sdb-seagate/graduacao/datasets/projeto_cerrado/2025-06-16")

ndvi_output_dir = batch_path / "ndvi-imgs"
ndre_output_dir = batch_path / "ndre-imgs"

fused_ndvi_output_dir = batch_path / "fused-ndvi-imgs"
fused_ndre_output_dir = batch_path / "fused-ndre-imgs"

rgb_ndvi_output_dir = batch_path / "rgb-ndvi-imgs"
rgb_ndre_output_dir = batch_path / "rgb-ndre-imgs"

ndvi_output_dir.mkdir(parents=True, exist_ok=True)
ndre_output_dir.mkdir(parents=True, exist_ok=True)

rgb_ndvi_output_dir.mkdir(parents=True, exist_ok=True)
rgb_ndre_output_dir.mkdir(parents=True, exist_ok=True)

fused_ndvi_output_dir.mkdir(parents=True, exist_ok=True)
fused_ndre_output_dir.mkdir(parents=True, exist_ok=True)

In [ ]:
types = ('*.JPG', '*.TIF')  # the tuple of file types
files_grabbed = []
for files in types:
    search_path = str(batch_path / files)
    files_grabbed.extend(glob.glob(search_path))

input_imgs = sorted(files_grabbed)

i = cycle(input_imgs)
slc = 6
total_batches = math.ceil(len(input_imgs) / slc)

print(f"Processando {total_batches} grupos de imagens...")
count = 0
for _ in range(math.ceil(len(input_imgs)/slc)):
    cur_imgs = list(islice(i, slc))

    rgb_path = Path(cur_imgs[0])
    fused_path = Path(cur_imgs[1])
    nir_path = Path(cur_imgs[3])
    red_path = Path(cur_imgs[4])
    re_path = Path(cur_imgs[5])

    try:
        rgb_image = np.asarray(Image.open(str(rgb_path)))
        fused_image = np.asarray(Image.open(str(fused_path)))

        # Calcular NDVI e NDRE
        ndvi_image = calculate_ndvi(str(red_path), str(nir_path))
        ndre_image = calculate_ndre(str(re_path), str(nir_path))
        
        # Stack com RGB
        stack_rgb_ndvi = np.dstack((rgb_image, ndvi_image))
        stack_rgb_ndre = np.dstack((rgb_image, ndre_image))

        # Stack com Fusão
        stack_fused_ndvi = np.dstack((fused_image, ndvi_image))
        stack_fused_ndre = np.dstack((fused_image, ndre_image))

        stem_name = rgb_path.stem 
        
        # Salvar NDVI e NDRE
        new_stem_ndvi = stem_name.replace('_D', '_NDVI')
        new_stem_ndre = stem_name.replace('_D', '_NDRE')

        ndvi_filename = f"{new_stem_ndvi}.TIF"
        ndre_filename = f"{new_stem_ndre}.TIF"

        tiff.imwrite(ndvi_output_dir / ndvi_filename, ndvi_image)
        tiff.imwrite(ndre_output_dir / ndre_filename, ndre_image)

        # Salvar stack com RGB
        new_stem_rgb_ndvi = stem_name.replace('_D', '_RGB_NDVI')
        new_stem_rgb_ndre = stem_name.replace('_D', '_RGB_NDRE')

        rgb_ndvi_filename = f"{new_stem_rgb_ndvi}.TIF"
        rgb_ndre_filename = f"{new_stem_rgb_ndre}.TIF"

        tiff.imwrite(rgb_ndvi_output_dir / rgb_ndvi_filename, stack_rgb_ndvi)
        tiff.imwrite(rgb_ndre_output_dir / rgb_ndre_filename, stack_rgb_ndre)

        # Salvar fusão
        new_stem_fused_ndvi = stem_name.replace('_D', '_FUSED_NDVI')
        new_stem_fused_ndre = stem_name.replace('_D', '_FUSED_NDRE')

        fused_ndvi_filename = f"{new_stem_fused_ndvi}.TIF"
        fused_ndre_filename = f"{new_stem_fused_ndre}.TIF"

        tiff.imwrite(fused_ndvi_output_dir / fused_ndvi_filename, stack_fused_ndvi)
        tiff.imwrite(fused_ndre_output_dir / fused_ndre_filename, stack_fused_ndre)

        count += 1
                
        if count % 10 == 0:
            print(f"\nProcessado: {count}/{total_batches} (Último: {rgb_path.name})")
            
    except Exception as e:
        print(f"Erro ao processar lote {count}: {e}")

print(f"\nCONCLUÍDO! {count} imagens geradas em 'ndvi-imgs' e 'ndre-imgs'.")
    